# Example sample-wise adaptation using OTDA and BOTDA

In [1]:
import numpy as np # always need it
from numpy import unravel_index
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import ot
import scipy.io
import mne          
from mne.decoding import CSP
mne.set_log_level(verbose='warning') #to avoid info at terminal
import matplotlib.pyplot as pl
from MIOTDAfunctions import*

In [2]:
rango_cl=[0.1, 1, 10]
rango_e=[0.1, 1, 10] 

## load data at filter it

In [28]:
fName = 'Data/DataSession1_S2.mat'
s = scipy.io.loadmat(fName)

Data_S1=s["X"]
Labels_S1=s["y"]
Labels_S1=np.squeeze(Labels_S1)

#filterting with mne
[nt, nc, ns]=np.shape(Data_S1)
Data_S1=np.reshape(Data_S1, [nt, nc*ns])
Data_S1=mne.filter.filter_data(Data_S1, 128, 8, 30)
Data_S1=np.reshape(Data_S1, [nt,nc,ns])

fName = 'Data/DataSession2_S2.mat'
s2 = scipy.io.loadmat(fName)

Data_S2=s2["X"]
Labels_S2=s2["y"]
Labels_S2=np.squeeze(Labels_S2)

#filterting with mne
[nt, nc, ns]=np.shape(Data_S2)
Data_S2=np.reshape(Data_S2, [nt, nc*ns])
Data_S2=mne.filter.filter_data(Data_S2, 128, 8, 30)
Data_S2=np.reshape(Data_S2, [nt,nc,ns])

### learn CSP+LDA from source data (Data_S1)

In [29]:
Xtr=Data_S1
Ytr=Labels_S1
csp = CSP(n_components=6, reg='empirical', log=True, norm_trace=False, cov_est='epoch')
#learn csp filters
Gtr=csp.fit_transform(Xtr,Ytr)
#learn lda
lda = LinearDiscriminantAnalysis()
lda.fit(Gtr,Ytr)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

### first 20 trials of the new session used as transportation set

In [30]:
##
Labels_te=Labels_S2[20:]
##
Xval=Data_S2[0:20]
Yval=Labels_S2[0:20]
##
Gval=csp.transform(Xval)

#### for saving outcomes

In [31]:
yt_predict_sc=[]
yt_predict_sr=[]
yt_predict_1=[]
yt_predict_2=[]
yt_predict_3=[]
yt_predict_4=[]

### select subset Gtr

In [32]:
# Subset selection re-training path
M=20
G_FOTDAs_, Y_FOTDAs_, regu_FOTDAs_=\
SelectSubsetTraining_OTDAs(xs=Gtr, ys=Ytr, xv=Gval, yv=Yval, rango_e=rango_e, metrica="euclidean", kfold=20, M=M, trad=True)
G_FOTDAl1l2_, Y_FOTDAl1l2_, regu_FOTDAl1l2_=\
    SelectSubsetTraining_OTDAl1l2(xs=Gtr, ys=Ytr, xv=Gval, yv=Yval, rango_e=rango_e, rango_cl=rango_cl, metrica="euclidean", kfold=20, M=M, trad=True)
G_BOTDAs_, Y_BOTDAs_, regu_BOTDAs_=\
    SelectSubsetTraining_BOTDAs(xs=Gtr, ys=Ytr, xv=Gval, yv=Yval, rango_e=rango_e, metrica="euclidean", kfold=20, M=M, trad=False)
G_BOTDAl1l2_, Y_BOTDAl1l2_, regu_BOTDAl1l2_=\
    SelectSubsetTraining_BOTDAl1l2(xs=Gtr, ys=Ytr, xv=Gval, yv=Yval, rango_e=rango_e, rango_cl=rango_cl, metrica="euclidean", kfold=20, M=M, trad=False)


## for each trial 

In [33]:
for re in range(1,len(Labels_te)+1):
    if np.mod(re,10)==0 : print('Running testing trial={:1.0f}'.format(re))
    #testing trial
    Xte=Data_S2[20+(re-1):20+(re)]
    Xte=Xte.reshape(1, nc, ns)
    Yte=Labels_S2[20+(re-1):20+(re)]
    
    Xval=np.vstack((Xval, Xte))
    Yval=np.hstack((Yval, Yte))
    
    #csp estimation
    Gval=csp.transform(Xval)
    Gte=csp.transform(Xte)
    #feature computation
    Gte=csp.transform(Xte)
    
    #evaluate SC  
    yt_predict_sc.append(lda.predict(Gte))

    #evaluate SR
    Xtr2=np.vstack((Xtr,Xval))
    Ytr2=np.hstack((Ytr, Yval))
    Xtr2=Xtr2[len(Yval):]
    Ytr2=Ytr2[len(Yval):]

    csp2 = CSP(n_components=6, reg='empirical', log=True, norm_trace=False, cov_est='epoch')
    #learn csp filters
    Gtr2=csp2.fit_transform(Xtr2,Ytr2)
    #learn lda
    lda2 = LinearDiscriminantAnalysis()
    lda2.fit(Gtr2,Ytr2)

    Gte2=csp2.transform(Xte)

    #ldatest
    yt_predict_sr.append(lda2.predict(Gte2))

    #%% # Sinkhorn Transport
    
    Gtr_daot=G_FOTDAs_
    Ytr_daot=Y_FOTDAs_ 
    ot_sinkhorn= ot.da.SinkhornTransport(metric="euclidean",reg_e=rango_e[regu_FOTDAs_])
    #learn the map
    ot_sinkhorn.fit(Xs=Gtr_daot, ys=Ytr_daot, Xt=Gval)
    #apply the mapping over source data
    transp_Xs_sinkhorn = ot_sinkhorn.transform(Xs=Gtr)

    # retraining
    lda3 = LinearDiscriminantAnalysis()
    lda3.fit(transp_Xs_sinkhorn,Ytr)
    # Compute acc
    yt_predict_1.append(lda3.predict(Gte))
    
       
    #%% # Group-Lasso Transport
  
    Gtr_daot=G_FOTDAl1l2_
    Ytr_daot=Y_FOTDAl1l2_
    
    ot_l1l2 = ot.da.SinkhornL1l2Transport(metric="euclidean",reg_e=rango_e[regu_FOTDAl1l2_[0]], reg_cl=rango_cl[regu_FOTDAl1l2_[1]])

    ot_l1l2.fit(Xs=Gtr_daot, ys=Ytr_daot, Xt=Gval)

    #transport taget samples onto source samples
    transp_Xs_l1l2=ot_l1l2.transform(Xs=Gtr)

    # retraining
    lda3 = LinearDiscriminantAnalysis()
    lda3.fit(transp_Xs_l1l2,Ytr)

    # Compute acc
    yt_predict_2.append(lda3.predict(Gte))
    
      
    #%% # Backward Sinkhorn Transport
       
    Gtr_botda=G_BOTDAs_
    Ytr_botda=Y_BOTDAs_
    
    bot_s = ot.da.SinkhornTransport(metric="euclidean",reg_e=rango_e[regu_BOTDAs_])
    
    
    bot_s.fit(Xs=Gval, ys=Yval, Xt=Gtr_botda)
    #transport testing samples
    transp_Xt_s_backward=bot_s.transform(Xs=Gte)
    # Compute accuracy one-training    
    yt_predict_3.append(lda.predict(transp_Xt_s_backward))
   
    #%% # Backward Group-Lasso Transport
       
    Gtr_botda=G_BOTDAl1l2_
    Ytr_botda=Y_BOTDAl1l2_
    
    bot_l1l2 = ot.da.SinkhornL1l2Transport(metric="euclidean",reg_e=rango_e[regu_BOTDAl1l2_[0]], reg_cl=rango_cl[regu_BOTDAl1l2_[1]])
        
    bot_l1l2.fit(Xs=Gval, ys=Yval, Xt=Gtr_botda)
    #transport testing samples
    transp_Xt_l1l2_backward=bot_l1l2.transform(Xs=Gte)
    # Compute accuracy one-training    
    yt_predict_4.append(lda.predict(transp_Xt_l1l2_backward))


Running testing trial=10
Running testing trial=20
Running testing trial=30
Running testing trial=40
Running testing trial=50
Running testing trial=60
Running testing trial=70
Running testing trial=80
Running testing trial=90
Running testing trial=100
Running testing trial=110
Running testing trial=120
Running testing trial=130
Running testing trial=140


In [27]:
# compute accuracy 
yt_predict_4=np.squeeze(np.asarray(yt_predict_4))
yt_predict_3=np.squeeze(np.asarray(yt_predict_3))
yt_predict_2=np.squeeze(np.asarray(yt_predict_2))
yt_predict_1=np.squeeze(np.asarray(yt_predict_1))
yt_predict_sc=np.squeeze(np.asarray(yt_predict_sc))
yt_predict_sr=np.squeeze(np.asarray(yt_predict_sr))

acc_botdal1l2=accuracy_score(Labels_te, yt_predict_4)
acc_botdas=accuracy_score(Labels_te, yt_predict_3)
acc_fotdal1l2=accuracy_score(Labels_te, yt_predict_2)
acc_fotdas=accuracy_score(Labels_te, yt_predict_1)
acc_sc=accuracy_score(Labels_te, yt_predict_sc)
acc_sr=accuracy_score(Labels_te, yt_predict_sr)

#print accuracy
acc={}
acc["sc"]=acc_sc
acc["sr"]=acc_sr
acc["fotda_s"]=acc_fotdas
acc["fotda_l1l2"]=acc_fotdal1l2
acc["botda_s"]=acc_botdas
acc["botda_l1l2"]=acc_botdal1l2
    
print(acc)  

{'sc': 0.7785714285714286, 'sr': 0.8928571428571429, 'fotda_s': 0.7214285714285714, 'fotda_l1l2': 0.7785714285714286, 'botda_s': 0.7, 'botda_l1l2': 1.0}
